<a href="https://colab.research.google.com/github/kridtapon/WFO-ChaosVortex-Strategy/blob/main/WFO_ChaosVortex_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate the Vortex Indicator (VI+ and VI-)
def calculate_vortex(data, period):
    """
    Calculate the Vortex Indicator (VI+ and VI-).
    """
    high_diff = data['High'].diff()
    low_diff = data['Low'].diff()

    # True range
    tr = pd.concat([data['High'] - data['Low'],
                    (data['High'] - data['Close'].shift()).abs(),
                    (data['Low'] - data['Close'].shift()).abs()], axis=1).max(axis=1)

    # Vortex Indicator (+ and -)
    vi_plus = (high_diff.where(high_diff > 0, 0).rolling(window=period).sum() / tr.rolling(window=period).sum()) * 100
    vi_minus = (low_diff.where(low_diff > 0, 0).rolling(window=period).sum() / tr.rolling(window=period).sum()) * 100
    return vi_plus, vi_minus

# Function to calculate Fractal Chaos Bands (Upper Band and Lower Band)
def calculate_fractal_bands(data, period):
    """
    Calculate Fractal Chaos Bands (Upper Band and Lower Band).
    """
    upper_band = data['Close'].rolling(window=period).max()
    lower_band = data['Close'].rolling(window=period).min()
    return upper_band, lower_band

# Walk-forward optimization with dynamic parameter ranges using Vortex and Fractal Bands
def walk_forward_optimization(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    vortex_period_range = range(5, 31)
    fractal_period_range = range(5, 31)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Use itertools to loop through all combinations dynamically
        for params in itertools.product(vortex_period_range, fractal_period_range):
            vortex_period, fractal_period = params

            # Calculate indicators on the training data
            train_data['VI+'], train_data['VI-'] = calculate_vortex(train_data, vortex_period)
            train_data['UB'], train_data['LB'] = calculate_fractal_bands(train_data, fractal_period)

            # Generate entry and exit signals
            entries = (
                (train_data['Close'] < train_data['UB']) &  # Close is below Upper Band
                (train_data['VI+'] > train_data['VI-'])    # VI+ is above VI-
            )
            exits = (
                (train_data['Close'] > train_data['LB']) &  # Close is above Lower Band
                (train_data['VI-'] > train_data['VI+'])    # VI- is above VI+
            )

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (vortex_period, fractal_period)

        # Test with the best parameters on the test data
        test_data['VI+'], test_data['VI-'] = calculate_vortex(test_data, best_params[0])
        test_data['UB'], test_data['LB'] = calculate_fractal_bands(test_data, best_params[1])

        entries = (
            (test_data['Close'] < test_data['UB']) &  # Close is below Upper Band
            (test_data['VI+'] > test_data['VI-'])    # VI+ is above VI-
        )
        exits = (
            (test_data['Close'] > test_data['LB']) &  # Close is above Lower Band
            (test_data['VI-'] > test_data['VI+'])    # VI- is above VI+
        )

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Define the stock symbol and time period
symbol = 'FICO'
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Perform walk-forward optimization
results = walk_forward_optimization(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply Vortex and Fractal Bands indicators
    yearly_data['VI+'], yearly_data['VI-'] = calculate_vortex(yearly_data, params[0])
    yearly_data['UB'], yearly_data['LB'] = calculate_fractal_bands(yearly_data, params[1])

    # Define entry/exit conditions
    entries = (yearly_data['Close'] < yearly_data['UB']) & (yearly_data['VI+'] > yearly_data['VI-'])
    exits = (yearly_data['Close'] > yearly_data['LB']) & (yearly_data['VI-'] > yearly_data['VI+'])

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
<ipython-input-3-b2de304a313c>:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-b2de304a313c>:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-b2de304a313c>:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return


Walk-Forward Optimization Results:
   Year Best_Params
0  2020    (14, 28)
1  2021    (14, 25)
2  2022    (15, 27)
3  2023    (15, 15)
4  2024    (15, 10)
5  2025     (7, 28)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           513861.100255
Total Return [%]                         413.8611
Benchmark Return [%]                   419.933663
Max Gross Exposure [%]                      100.0
Total Fees Paid                      29051.227304
Max Drawdown [%]                        21.782045
Max Drawdown Duration                       104.0
Total Trades                                   64
Total Closed Trades                            64
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                              67.1875
Best Trade [%]          

<ipython-input-3-b2de304a313c>:140: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-b2de304a313c>:140: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-b2de304a313c>:141: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-b2de3

In [4]:
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,519933.663401
Total Return [%],419.933663
Benchmark Return [%],419.933663
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],50.900922
